In [5]:
from openai import OpenAI
import pandas as pd
import json

In [11]:
client = OpenAI()

In [4]:
get_muscle = """
You are a professional exercise physiologist, expertise in workout knowledge and human body muscle groups. \
Let's proceed step by step. \
Kindly furnish a list listing every possible muscle groups involved. \
Organize the list from large to small, based on the provided equipment. \
Please conclude the file with the marker [end]. \
For example,
Muscle Group:
Hamstrings
Gluteus Maximus
Erector Spinae
Gastrocnemius
Soleus
Adductor Magnus
Semimembranosus
Semitendinosus
Biceps Femoris
Popliteus
Gracilis
[end]

Now, given equipment

"""

In [30]:
def chat(prompt, topic):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content":f"""{prompt}{topic}."""}
          ]
    )
    return completion.choices[0].message.content


## Get muscle groups using ChatGPT

In [87]:
test = chat(get_muscle, "leg curl")

In [89]:
test

'Muscle Group:\nHamstrings\nBiceps Femoris\nSemimembranosus\nSemitendinosus\nGracilis\nAdductor Magnus\nPopliteus\nGastrocnemius\nSoleus\nGluteus Maximus\nErector Spinae\n[end]'

In [91]:
sp = test.split("\n")
sp

['Muscle Group:',
 'Hamstrings',
 'Biceps Femoris',
 'Semimembranosus',
 'Semitendinosus',
 'Gracilis',
 'Adductor Magnus',
 'Popliteus',
 'Gastrocnemius',
 'Soleus',
 'Gluteus Maximus',
 'Erector Spinae',
 '[end]']

In [6]:
equipment = pd.read_csv("equipment.csv")

In [7]:
equipment.head()

,id,equipment,number,capacity,muscle_groups,remark
0,0,treadmill,9,1,NaN,NaN
1,1,rowing machine,4,1,NaN,NaN
2,2,upright bike,2,1,NaN,NaN
3,3,elliptical machine,3,1,NaN,NaN
4,4,bench,7,1,NaN,NaN


In [8]:
muscle_groups = []

```{python}
# count = 0
for item in equipment.loc[:, 'equipment']:
  # if count < 5:
    muscle_groups.append(getMuscleGroup(item))
    # count += 1
```

```{python}
with open("data/muscle_groups_ori.txt", "w") as f:
  for t in muscle_groups:
    f.write(f"{t}\n")
```

In [ ]:
muscle_groups

# Data Processing

### Counting
Count the number of all muscle groups and translate the names into Traditional Chinese.

In [52]:
muscle_groups_all = []
muscle_dict = dict()
for i in range(len(muscle_groups)):
  # num_split = 0
  # num_group = 0
  muscles = []
  # print(i)
  for muscle in muscle_groups[i].split("\n"):
    # num_split += 1
    if not (muscle.startswith("Muscle Group") or muscle == ''):
      if muscle == "[end]":
        # print(f"END:DDD, {muscle}")
        break
      # num_group += 1
      if muscle.split(". ")[0].isdigit():
        muscle = muscle.split(". ")[1]
      elif muscle.startswith("- "):
        muscle = muscle.split("- ")[1]
      # print(muscle)
      muscles.append(muscle)
      muscle_groups_all.append(muscle)
  muscle_dict[i] = muscles

In [53]:
len(muscle_groups_all)

485

```{python}
# Convert and write JSON object to file
with open("data/muscle_dict.json", "w") as outfile:
    json.dump(muscle_dict, outfile)
```

In [122]:
with open("data/muscle_dict.json", "r") as f:
  muscle_dict = json.load(f)

In [ ]:
muscle_dict

In [131]:
# flatten the lists and get unique items
unique_items = list(set(item for sublist in muscle_dict.values() for item in sublist))

In [133]:
unique_items.sort()

In [134]:
unique_items

['Adductor Brevis',
 'Adductor Longus',
 'Adductor Magnus',
 'Anconeus',
 'Anterior Deltoid',
 'Anterior Deltoids (Front shoulder)',
 'Biceps',
 'Biceps Brachii',
 'Biceps Femoris',
 'Biceps brachii',
 'Brachialis',
 'Brachioradialis',
 'Calf Muscles',
 'Calves',
 'Calves (Gastrocnemius and Soleus)',
 'Calves (Gastrocnemius, Soleus)',
 'Coracobrachialis',
 'Deltoid',
 'Deltoids',
 'Deltoids (Anterior, Lateral, and Posterior)',
 'Deltoids (Anterior, Middle, Posterior)',
 'Deltoids (Posterior)',
 'Deltoids (Rear)',
 'Erector Spinae',
 'Erector Spinae (Spinalis)',
 'Extensor Digitorum Longus',
 'Extensor Hallucis Longus',
 'External Obliques',
 'Flexor Digitorum Longus',
 'Flexor Hallucis Longus',
 'Forearm muscles (Flexors and Extensors)',
 'Gastrocnemius',
 'Gastrocnemius (Lateral head)',
 'Gastrocnemius (Medial head)',
 'Gluteus Maximus',
 'Gluteus Medius',
 'Gluteus Minimus',
 'Gracilis',
 'Hamstrings',
 'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
 'Hip Abductors',

| Original | Replaced |
| --- | --- |
| Anterior Deltoid | Anterior Deltoids (Front shoulder) |
| Biceps | Biceps Brachii |
| Biceps brachii | Biceps Brachii |
| Calves | Calves (Gastrocnemius and Soleus) |
| Calves (Gastrocnemius, Soleus) | Calves (Gastrocnemius and Soleus) |
| Deltoid | Deltoids (Anterior, Lateral, and Posterior) |
| Deltoids | Deltoids (Anterior, Lateral, and Posterior)
| Deltoids (Anterior, Middle, Posterior) | Deltoids (Anterior, Lateral, and Posterior) |
| Hamstrings | Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus) |
| Rear Deltoids | Posterior Deltoid |
| Deltoids (Posterior) | Posterior Deltoid |
| Deltoids (Rear) | Posterior Deltoid |
| Medial Deltoid | Lateral Deltoid |
| Pectoralis Major (Upper chest) | Pectoralis Major (Clavicular Head) |
| Serratus Anterior | Serratus Anterior (Upper side of ribs) |
| Terres Major | Teres Major |
| Transversus Abdominis | Transverse Abdominis |
| Triceps | Triceps Brachii (Back of upper arm) |
| Triceps Brachii | Triceps Brachii (Back of upper arm) |
| Trapezius (Middle and Lower) | Trapezius (middle and lower fibers) |
| Quadriceps | Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius) |
| Rhomboideus | Rhomboids |

### Replacement

In [184]:
muscle_dict_rep = muscle_dict.copy()

In [185]:
# replacement table
replacement_table = {
    'Anterior Deltoid': 'Anterior Deltoids (Front shoulder)',
    'Biceps': 'Biceps Brachii',
    'Biceps brachii': 'Biceps Brachii',
    'Calves': 'Calves (Gastrocnemius and Soleus)',
    'Calves (Gastrocnemius, Soleus)': 'Calves (Gastrocnemius and Soleus)',
    'Deltoid': 'Deltoids (Anterior, Lateral, and Posterior)',
    'Deltoids': 'Deltoids (Anterior, Lateral, and Posterior)',
    'Deltoids (Anterior, Middle, Posterior)': 'Deltoids (Anterior, Lateral, and Posterior)',
    'Hamstrings': 'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
    'Rear Deltoids': 'Posterior Deltoid',
    'Deltoids (Posterior)': 'Posterior Deltoid',
    'Deltoids (Rear)': 'Posterior Deltoid',
    'Medial Deltoid': 'Lateral Deltoid',
    'Pectoralis Major (Upper chest)': 'Pectoralis Major (Clavicular Head)',
    'Serratus Anterior': 'Serratus Anterior (Upper side of ribs)',
    'Terres Major': 'Teres Major',
    'Transversus Abdominis': 'Transverse Abdominis',
    'Triceps': 'Triceps Brachii (Back of upper arm)',
    'Triceps Brachii': 'Triceps Brachii (Back of upper arm)',
    'Trapezius (Middle and Lower)': 'Trapezius (middle and lower fibers)',
    'Quadriceps': 'Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius)',
    'Rhomboideus': 'Rhomboids'
}

# function to replace values in a list using a mapping
def replaceValues(muscle_list, replacement_mapping):
    rep = [replacement_mapping.get(muscle, muscle) for muscle in muscle_list]
    return rep
    # return [print(replacement_mapping.get(muscle, muscle)) for muscle in muscle_list]

# Apply the replacement to each list in the dictionary
for key, muscles in muscle_dict_rep.items():
    # print(key)
    muscle_dict_rep[key] = replaceValues(muscles, replacement_table)

In [ ]:
muscle_dict_rep

In [186]:

# Convert and write JSON object to file
with open("data/muscle_dict_rep.json", "w") as outfile:
    json.dump(muscle_dict_rep, outfile)


In [144]:
with open("data/muscle_dict_rep.json", "r") as f:
  muscle_dict_rep = json.load(f)

In [187]:
unique_items = list(set(item for sublist in muscle_dict_rep.values() for item in sublist))
unique_items.sort()

In [197]:
len(unique_items)

81

In [188]:
unique_items

['Adductor Brevis',
 'Adductor Longus',
 'Adductor Magnus',
 'Anconeus',
 'Anterior Deltoids (Front shoulder)',
 'Biceps Brachii',
 'Biceps Femoris',
 'Brachialis',
 'Brachioradialis',
 'Calf Muscles',
 'Calves (Gastrocnemius and Soleus)',
 'Coracobrachialis',
 'Deltoids (Anterior, Lateral, and Posterior)',
 'Erector Spinae',
 'Erector Spinae (Spinalis)',
 'Extensor Digitorum Longus',
 'Extensor Hallucis Longus',
 'External Obliques',
 'Flexor Digitorum Longus',
 'Flexor Hallucis Longus',
 'Forearm muscles (Flexors and Extensors)',
 'Gastrocnemius',
 'Gastrocnemius (Lateral head)',
 'Gastrocnemius (Medial head)',
 'Gluteus Maximus',
 'Gluteus Medius',
 'Gluteus Minimus',
 'Gracilis',
 'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
 'Hip Abductors',
 'Hip Adductors',
 'Hip Flexors',
 'Iliopsoas',
 'Infraspinatus',
 'Internal Obliques',
 'Internal and External Intercostals',
 'Internal and External Obliques',
 'Lateral Deltoid',
 'Latissimus Dorsi',
 'Levator Scapulae',


### Adding

In [182]:
# muscle_dict_rep_add = muscle_dict_rep.copy()

In [183]:
"""# adding table
add_table = {
    'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)': 'Hamstrings',
    'Pectoralis Major (Clavicular Head)': 'Pectoralis Major',
    'Pectoralis Major (Sternal Head)': 'Pectoralis Major',
    'Pectoralis Major (Upper chest)': 'Pectoralis Major',
    'Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius)': 'Quadriceps',
    'Rhomboids': 'Rhomboideus',
    'Rhomboideus': 'Rhomboids',
    'Serratus Anterior (Upper side of ribs)': 'Serratus Anterior',
    'Trapezius (Middle and Lower)': 'Trapezius',
    'Trapezius (lower fibers)': 'Trapezius',
    'Trapezius (middle and lower fibers)': 'Trapezius',
    'Triceps Brachii (Back of upper arm)': 'Triceps Brachii',
    'Triceps Brachii (Long Head)': 'Triceps Brachii',
}

# update values in a list using a mapping
for key, muscles in muscle_dict_rep_add.items():
    # print(key)
    add_list = []
    for muscle in muscles:
        add = add_table.get(muscle)
        if add is not None:
            add_list.append(add)
    # print(add_list)
    if add_list != []:
        muscle_dict_rep_add[key] += add_list"""

In [ ]:
# muscle_dict_rep_add

In [ ]:
# Convert and write JSON object to file
# with open("data/muscle_dict_rep_add.json", "w") as outfile:
#     json.dump(muscle_dict_rep_add, outfile)

In [6]:
# with open("data/muscle_dict_rep_add.json", "r") as f:
#   muscle_dict_rep_add = json.load(f)

In [21]:
# flatten the lists and get unique items
# unique_items = list(set(item for sublist in muscle_dict_rep_add.values() for item in sublist))

In [ ]:
# unique_items

In [ ]:
# len(unique_items)

### Categorization

In [192]:
categorize ="""
You are a professional exercise physiologist, expertise in workout knowledge and human body muscle groups. \
Given 6 categories: arm, abdominal, shoulder, chest, back, and leg. \
Provide the category only. \
Example 1:
Rhomboid Minor
Back

Example 2:
Anterior Deltoids
Shoulder

Now, Categorize the following human muscle group:
"""

In [193]:
test = chat(categorize, unique_items[0])

In [194]:
test

'Leg'

In [195]:
category = dict()
count = 0
for name in unique_items:
  # if count < 5:
    category[name] = chat(categorize, name)
    # count += 1

In [196]:
category

{'Adductor Brevis': 'Leg',
 'Adductor Longus': 'Leg',
 'Adductor Magnus': 'Leg',
 'Anconeus': 'Arm',
 'Anterior Deltoids (Front shoulder)': 'Shoulder',
 'Biceps Brachii': 'Arm',
 'Biceps Femoris': 'Leg',
 'Brachialis': 'Arm',
 'Brachioradialis': 'Arm',
 'Calf Muscles': 'Leg',
 'Calves (Gastrocnemius and Soleus)': 'Leg',
 'Coracobrachialis': 'Arm',
 'Deltoids (Anterior, Lateral, and Posterior)': 'Shoulder',
 'Erector Spinae': 'Back',
 'Erector Spinae (Spinalis)': 'Back',
 'Extensor Digitorum Longus': 'Leg',
 'Extensor Hallucis Longus': 'Leg',
 'External Obliques': 'Abdominal',
 'Flexor Digitorum Longus': 'Leg',
 'Flexor Hallucis Longus': 'Leg',
 'Forearm muscles (Flexors and Extensors)': 'Arm',
 'Gastrocnemius': 'Leg',
 'Gastrocnemius (Lateral head)': 'Leg',
 'Gastrocnemius (Medial head)': 'Leg',
 'Gluteus Maximus': 'Leg',
 'Gluteus Medius': 'Leg',
 'Gluteus Minimus': 'Leg',
 'Gracilis': 'Leg',
 'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)': 'Leg',
 'Hip Abductors': 'Le

In [198]:
# Convert and write JSON object to file
with open("data/category.json", "w") as outfile:
    json.dump(category, outfile)

In [203]:
with open("data/category.json", "r") as f:
  category = json.load(f)

In [204]:
def reverseDict(dic, value):
  v_list =[]
  for key in dic:
    if dic[key] == value:
      v_list.append(key)
  return v_list

In [205]:
category_rev = dict()

In [206]:
for c in category.values():
  c_list = reverseDict(category, c)
  category_rev[c] = c_list

In [207]:
category_rev

{'Leg': ['Adductor Brevis',
  'Adductor Longus',
  'Adductor Magnus',
  'Biceps Femoris',
  'Calf Muscles',
  'Calves (Gastrocnemius and Soleus)',
  'Extensor Digitorum Longus',
  'Extensor Hallucis Longus',
  'Flexor Digitorum Longus',
  'Flexor Hallucis Longus',
  'Gastrocnemius',
  'Gastrocnemius (Lateral head)',
  'Gastrocnemius (Medial head)',
  'Gluteus Maximus',
  'Gluteus Medius',
  'Gluteus Minimus',
  'Gracilis',
  'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
  'Hip Abductors',
  'Hip Adductors',
  'Hip Flexors',
  'Iliopsoas',
  'Peroneals',
  'Peroneus Brevis',
  'Peroneus Longus',
  'Peroneus Tertius',
  'Plantaris',
  'Popliteus',
  'Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius)',
  'Rectus Femoris',
  'Sartorius',
  'Semimembranosus',
  'Semitendinosus',
  'Soleus',
  'Tensor Fasciae Latae',
  'Tibialis Anterior',
  'Vastus Intermedius',
  'Vastus Lateralis',
  'Vastus Medialis'],
 'Arm': ['Anconeus',
  'Biceps Brac

In [208]:
with open("data/category_rev.json", "w") as outfile:
    json.dump(category_rev, outfile)

Add categories to `muscle_dict_rep_add`

In [209]:
muscle_dict_comple = muscle_dict_rep.copy()

In [210]:
def addCategory(muscles):
  add_list = []
  for muscle in muscles:
    if category[muscle] not in add_list:
      add_list.append(category[muscle])
  return add_list

In [211]:
for key, muscles in muscle_dict_comple.items():
  add_list = addCategory(muscles)
  if add_list != []:
    muscle_dict_comple[key] += add_list

In [ ]:
muscle_dict_comple

In [213]:
with open("data/muscle_dict_comple.json", "w") as outfile:
    json.dump(muscle_dict_comple, outfile)

In [214]:
unique_items = list(set(item for sublist in muscle_dict_comple.values() for item in sublist))
unique_items.sort()

In [216]:
len(unique_items)

87

In [ ]:
unique_items

## Translation
EN -> TC

In [217]:
traslate ="""
You are a professional exercise physiologist, expertise in workout knowledge and human body muscle groups. \
Translate the following muscle group into TRADITIONAL Chinese (Taiwan). \
Exclude phonetic symbols. \
Simply provide the translated word. \
Example 1:
Biceps
二頭肌

Example 2:
Biceps Femoris
股二頭肌

Now, translate this muscle group:
"""

In [218]:
unique_items[0]

'Abdominal'

In [219]:
test = chat(traslate, unique_items[0])

In [220]:
test

'腹肌'

In [221]:
translate_dict = dict()
# count = 0
for name in unique_items:
  # if count < 5:
    # print(name)
    translate_dict[chat(traslate, name)] = name
    # count += 1

In [222]:
translate_dict

{'腹肌': 'Abdominal',
 '內收肌短肌': 'Adductor Brevis',
 '內收肌': 'Adductor Longus',
 '腹股沉肌': 'Adductor Magnus',
 '肘肌': 'Anconeus',
 '前三角肌': 'Anterior Deltoids (Front shoulder)',
 '手臂': 'Arm',
 '背肌': 'Back',
 '肱二頭肌': 'Biceps Brachii',
 '半腱肌': 'Semitendinosus',
 '上臂三頭肌': 'Brachialis',
 '肱桡肌': 'Brachioradialis',
 '腓腸肌': 'Peroneus Longus',
 '小腿肌肉 (腓腸肌和腓腸後肌)': 'Calves (Gastrocnemius and Soleus)',
 '胸肌': 'Pectoralis Major',
 '肩胛肌': 'Coracobrachialis',
 '三角肌（前束、外束、後束）': 'Deltoids (Anterior, Lateral, and Posterior)',
 '脊柱舉起肌': 'Erector Spinae',
 '脊柱起肌 (脊肌)': 'Erector Spinae (Spinalis)',
 '胫骨長趾伸肌': 'Extensor Digitorum Longus',
 '長趾伸肌': 'Extensor Hallucis Longus',
 '外腹斜肌': 'External Obliques',
 '足長屈肌': 'Flexor Digitorum Longus',
 '屈長跗肌': 'Flexor Hallucis Longus',
 '前臂肌肉 (屈肌和伸肌)': 'Forearm muscles (Flexors and Extensors)',
 '腓腓肌': 'Gastrocnemius',
 '腓腸肌 (外側頭)': 'Gastrocnemius (Lateral head)',
 '腓腸肌（內側）': 'Gastrocnemius (Medial head)',
 '臀大肌': 'Gluteus Maximus',
 '中殿肌': 'Gluteus Medius',
 '臀小肌': 'Gluteus 

In [223]:
with open("data/translate_dict.json", "w") as outfile:
    json.dump(translate_dict, outfile)

# Update
Update muscle_groups info

In [143]:
equipment["muscle_groups"] = muscle_dict_comple

In [149]:
equipment

,id,equipment,number,capacity,muscle_groups,remark
0,0,treadmill,9,1,"[Quadriceps, Hamstrings, Gluteus Maximus, Gast...",NaN
1,1,rowing machine,4,1,"[Quadriceps, Hamstrings, Gluteus Maximus, Erec...",NaN
2,2,upright bike,2,1,"[Quadriceps, Gluteus Maximus, Hamstrings, Gast...",NaN
3,3,elliptical machine,3,1,"[Gluteus Maximus, Quadriceps, Hamstrings, Calv...",NaN
4,4,bench,7,1,"[Quadriceps, Hamstrings, Gluteus Maximus, Erec...",NaN
5,5,squat machine,1,1,"[Quadriceps, Gluteus Maximus, Hamstrings, Erec...",NaN
6,6,leg press and hack squat machine,1,1,"[Quadriceps, Gluteus Maximus, Hamstrings, Gast...",NaN
7,7,standing calf,1,1,"[Gastrocnemius, Soleus, Now, given equipment, ...",NaN
8,8,abdominal machine,1,1,"[Rectus Abdominis, Transversus Abdominis, Exte...",NaN
9,9,spin machine,5,1,"[Quadriceps, Hamstrings, Gluteus Maximus, Gast...",NaN


In [148]:
type(equipment.loc[:, "muscle_groups"][0])

list